In [176]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import re
import random
import time
import pandas as pd
import joblib
import numpy
import pytz
import scipy
import six
import sklearn
import soupsieve
import urllib3
import csv
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
from math import *
from operator import itemgetter
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

## [과제 1] 네이버 평점 데이터 수집

In [24]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)

In [3]:
# def get_user_list(limit):
#     page = 1
#     user_list = []
#     while len(user_list) <= limit:
#         driver.get('https://movie.naver.com/movie/point/af/list.nhn?&page={0}'.format(page))

#         html = driver.page_source
#         soup = BeautifulSoup(html, 'html.parser')
#         reviewNum = soup.select('#old_content > table > tbody > tr > td.ac.num')
#         userId = soup.select('table.list_netizen > tbody > tr > td > a.author')

#         for i in range(len(reviewNum)):
#             review = str(reviewNum[i]).replace('<td class="ac num">','').replace('</td>','')
#             user = userId[i].text.replace('*','')
#             if user not in [user[1] for user in user_list]:
#                 user_list.append((review, user))

#         time.sleep(random.randrange(2, 5))
#         page += 1

#     return user_list[:limit]

In [4]:
def get_movie_link(soup):
    movie_links = soup.select('a[href]')

    movie_links_list = []
    for link in movie_links:
        
        if re.search(r'st=mcode&amp;sword' and r'&target=after$', link['href']):
            target_url = 'https://movie.naver.com/movie/point/af/list.nhn'+str(link['href'])
            movie_links_list.append(target_url)

    return movie_links_list[1:]

In [25]:
def get_review(userid,reviewNo,page):
    user_reviews = []

    '''
    dirver & beautifulsoup init
    '''
    driver.get('https://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword={0}&target=after'.format(reviewNo)+'&page='+str(page))

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    
    # 영화 링크를 가져오는 코드
    movie_links = get_movie_link(soup)
    movieId = [link.replace('https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=','').replace('&target=after','') for link in movie_links]
    
    
    reviewNum = soup.select('#old_content > table > tbody > tr > td.point')
    review_list = soup.select('#old_content > table > tbody > tr > td.ac.num')
        
    for i in range(len(reviewNum)):
        review_max = str(review_list[i]).replace('<td class="ac num">','').replace('</td>','')
        if int(review_max) <= reviewNo:
            review = str(reviewNum[i]).replace('<td class="point">','').replace('</td>','')
            user_reviews.append(int(review))
        else:
            movieId.pop(0)
    print("MovieId : ", movieId)
    print("rating : ", user_reviews)
    
    
    # csv 파일에 row 추가
    f = open('rating2.csv', 'a', encoding='utf-8')
    wr = csv.writer(f)
    
    for j in range(len(movieId)):
        wr.writerow([userid, user_reviews[j], movieId[j]])
    f.close()
    
    # 다음 페이지 번호
    next_pagenum = soup.select('#old_content > div.paging > div > a.pg_next')
    
    # 다음 페이지가 있을 경우
    if next_pagenum:
        nextpage = (str(next_pagenum[0]).split('page=')[1])
        nextpage = nextpage.split('\"')[0]
        print(nextpage)
        
        nextlink = 'https://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword='+str(reviewNo)+'&target=after'+'&page='+str(nextpage)
        driver.get(nextlink)
        html2 = driver.page_source
        soup2 = BeautifulSoup(html2, 'html.parser')
        
        time.sleep(random.randrange(3, 5))
        
        get_review(userid,reviewNo,nextpage)
        
    # 다음 페이지가 없을 경우
    if not next_pagenum:
        return 0
    
    '''
    crawling information
    '''
    
#     time.sleep(random.randrange(3, 5))

    return 0

In [26]:
def main():
#     print(get_user_list(5))
    user = pd.read_csv('./data/naver_user.csv')
    for i in user.iterrows():
        userid = user[user.reviewNo == i[1]['reviewNo']]['userId'].iloc[0]
        get_review(userid,i[1]['reviewNo'],1)

#     driver.close()

In [28]:
if __name__ == "__main__":
    main()

MovieId :  ['136900', '163788', '174065']
rating :  [2, 10, 10]
MovieId :  ['154667', '136900', '145335', '120141', '54704']
rating :  [10, 10, 10, 9, 9]
MovieId :  ['113351', '86507', '39705', '39756', '31925', '35098', '168679', '123300', '151188']
rating :  [7, 10, 10, 1, 10, 6, 1, 8, 2]
2
MovieId :  ['41592', '18811', '118994', '164183', '76016', '130997', '24522', '148909', '139700', '158622']
rating :  [2, 10, 1, 2, 10, 10, 10, 1, 2, 1]
3
MovieId :  ['171725', '181411', '164932', '132623', '58072', '67696', '61060', '76037', '141104', '34431']
rating :  [10, 6, 7, 1, 10, 5, 1, 10, 6, 10]
4
MovieId :  ['38689', '31882', '158112', '126034', '116233', '167651', '167697', '67533', '105246', '146501']
rating :  [10, 10, 1, 1, 6, 7, 3, 5, 8, 6]
5
MovieId :  ['106065', '161868', '146485', '29558', '152632', '169643', '122596', '39907', '57971', '10504']
rating :  [6, 1, 2, 9, 9, 8, 9, 6, 8, 8]
6
MovieId :  ['13008', '87305', '37838', '29321', '151153', '11181', '165475', '170879', '2756

In [115]:
rating = pd.read_csv('./rating2.csv')
rating

,userId,rating,movieId
0,airf,2,136900
1,airf,10,163788
2,airf,10,174065
3,nanw,10,154667
4,nanw,10,136900
5,nanw,10,145335
6,nanw,9,120141
7,nanw,9,54704
8,zxcv,7,113351
9,zxcv,10,86507


In [116]:
rating = rating.drop_duplicates()

In [117]:
rating.to_csv('rating.csv',index=False)

In [118]:
rating = pd.read_csv('./rating.csv')
rating

,userId,rating,movieId
0,airf,2,136900
1,airf,10,163788
2,airf,10,174065
3,nanw,10,154667
4,nanw,10,136900
5,nanw,10,145335
6,nanw,9,120141
7,nanw,9,54704
8,zxcv,7,113351
9,zxcv,10,86507


In [119]:
user_num = rating.userId.unique().shape[0]
movie_num = rating.movieId.unique().shape[0]
print("유저의 수는 {}명 이고, \n영화의 수는 {}개 입니다.".format(user_num,movie_num))

유저의 수는 100명 이고, 
영화의 수는 2697개 입니다.


## [과제 2-a] 유사 사용자 탐색

In [120]:
user = pd.read_csv('./data/naver_user.csv')
user.head(5)

,reviewNo,userId
0,15772038,airf
1,15772037,nanw
2,15772036,zxcv
3,15772035,sdh1
4,15772032,guan


In [121]:
join = pd.merge(rating, user, how='inner')
join

,userId,rating,movieId,reviewNo
0,airf,2,136900,15772038
1,airf,10,163788,15772038
2,airf,10,174065,15772038
3,nanw,10,154667,15772037
4,nanw,10,136900,15772037
5,nanw,10,145335,15772037
6,nanw,9,120141,15772037
7,nanw,9,54704,15772037
8,zxcv,7,113351,15772036
9,zxcv,10,86507,15772036


In [122]:
group_user = join.groupby(['userId','reviewNo']).size().nlargest(10).reset_index()
group_user['userId'].iloc[0]

'ykm3'

In [123]:
join[join.userId == 'imag']

,userId,rating,movieId,reviewNo
3201,imag,10,161967,15771940
3202,imag,9,181409,15771940
3203,imag,9,52429,15771940
3204,imag,1,181003,15771940
3205,imag,1,173884,15771940
3206,imag,6,174835,15771940
3207,imag,8,144266,15771940
3208,imag,7,164106,15771940
3209,imag,8,132996,15771940
3210,imag,10,155356,15771940


In [124]:
joinf = open('top10.csv', 'w', encoding='utf-8')
wr = csv.writer(joinf)
wr.writerow(['userId','rating','movieId','reviewNo'])
for j in range(len(group_user)):
    wr.writerow(join[join.userId == group_user['userId'].iloc[j]].iloc[1])
joinf.close()

In [129]:
top10 = pd.read_csv('./top10.csv')
top10

,userId,rating,movieId,reviewNo
0,ykm3,3,145162,15771936
1,sang,10,161967,15771961
2,tsp0,7,163788,15771934
3,hosu,7,180399,15771998
4,zxcv,10,86507,15772036
5,zard,10,158653,15772012
6,artn,10,172174,15771948
7,suha,7,180399,15771976
8,ldsl,1,157297,15771977
9,imag,9,181409,15771940


In [135]:
UM_matrix_ds = join.pivot(index='reviewNo', columns='movieId', values='rating')
UM_matrix_ds.head(100)

movieId,10002,10003,10004,10005,10006,10008,10009,10012,10016,10018,...,181409,181410,181411,181414,181419,181711,182348,182360,183132,183877
reviewNo,,,,,,,,,,,,,,,,,,,,,
15771922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## [과제 2-a] 결과 - Cosine

In [137]:
def distance_cosine(a, b):
    return 1-distance.cosine(a, b)

In [253]:
def nearest_neighbor_user(user, topN, simFunc):
    ul = UM_matrix_ds.loc[user].dropna()
    ratedIndex = ul.index
    nn = {}
    
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = []
        interSectionU2 = []
        if uid == user: continue
            
        for i in ratedIndex:
            if not math.isnan(row[i]):
                interSectionU1.append(ul[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
            
        sim = round(simFunc(interSectionU1, interSectionU2),2)
        if not math.isnan(sim): nn[uid] = sim

            
    return sorted(nn.items(), key= itemgetter(1))[:-(topN+1):-1]

In [273]:
for reNo in top10['reviewNo']:
    print('User {0} neighbors : {1}'.format(reNo, (nearest_neighbor_user(int(reNo), 3, distance_cosine))))

User 15771936 neighbors : [(15771972, 1.0), (15771993, 0.97), (15772003, 0.95)]
User 15771961 neighbors : [(15771970, 1.0), (15771966, 1.0), (15771947, 1.0)]
User 15771934 neighbors : [(15772009, 1.0), (15771942, 1.0), (15772030, 0.99)]
User 15771998 neighbors : [(15771970, 1.0), (15771965, 1.0), (15771947, 1.0)]
User 15772036 neighbors : [(15771959, 1.0), (15772000, 0.98), (15771966, 0.98)]
User 15772012 neighbors : [(15772003, 1.0), (15772016, 0.95), (15772000, 0.92)]
User 15771948 neighbors : [(15771966, 1.0), (15771945, 1.0), (15771943, 0.98)]
User 15771976 neighbors : [(15771974, 1.0), (15771966, 1.0), (15771947, 1.0)]
User 15771977 neighbors : [(15771971, 1.0), (15771959, 1.0), (15771983, 0.99)]
User 15771940 neighbors : [(15771993, 1.0), (15771981, 1.0), (15771947, 1.0)]


## [과제 2-a] 결과 - Correlation

In [269]:
def distance_correlation(a,b):
    return 1-distance.correlation(a,b)

In [276]:
for reNo in top10['reviewNo']:
    print('User {0} neighbors : {1}'.format(reNo, (nearest_neighbor_user(int(reNo), 3, distance_correlation))))

User 15771936 neighbors : [(15771993, 1.0), (15772022, 0.69), (15772019, 0.5)]
User 15771961 neighbors : [(15771972, 1.0), (15772015, 0.94), (15771974, 0.93)]
User 15771934 neighbors : [(15771942, 1.0), (15772030, 0.98), (15771947, 0.96)]
User 15771998 neighbors : [(15772005, 0.89), (15771922, 0.88), (15771980, 0.66)]
User 15772036 neighbors : [(15771966, 0.97), (15771943, 0.94), (15771940, 0.6)]
User 15772012 neighbors : [(15772016, 0.94), (15771974, 0.58), (15771969, 0.58)]
User 15771948 neighbors : [(15771943, 0.94), (15771938, 0.73), (15771926, 0.59)]
User 15771976 neighbors : [(15771940, 0.85), (15771974, 0.58), (15771927, 0.53)]
User 15771977 neighbors : [(15771971, 1.0), (15771959, 1.0), (15771998, 0.66)]
User 15771940 neighbors : [(15771981, 0.98), (15771995, 0.97), (15771974, 0.94)]


## [과제 2-a] 결과 - Euclidean

In [257]:
def distance_euclidean(a,b):
    return 1 / (1+distance.euclidean(a,b))

In [260]:
for reNo in top10['reviewNo']:
    print('User {0} neighbors : {1}'.format(reNo, (nearest_neighbor_user(int(reNo), 3, distance_euclidean))))

User 15771936 neighbors : [(15771972, 0.25), (15771993, 0.2), (15772020, 0.12)]
User 15771961 neighbors : [(15772031, 0.33), (15771970, 0.33), (15771954, 0.33)]
User 15771934 neighbors : [(15772030, 0.41), (15771988, 0.29), (15772003, 0.25)]
User 15771998 neighbors : [(15771922, 0.17), (15772030, 0.16), (15772005, 0.15)]
User 15772036 neighbors : [(15771959, 1.0), (15771943, 0.24), (15772022, 0.19)]
User 15772012 neighbors : [(15772016, 0.22), (15772003, 0.15), (15771974, 0.13)]
User 15771948 neighbors : [(15771945, 0.41), (15771943, 0.23), (15771938, 0.15)]
User 15771976 neighbors : [(15771974, 0.31), (15771926, 0.31), (15771966, 0.29)]
User 15771977 neighbors : [(15771971, 1.0), (15771959, 0.5), (15771938, 0.25)]
User 15771940 neighbors : [(15771993, 1.0), (15771947, 1.0), (15771981, 0.33)]


## [과제 2-b] 영화 평점 예측

In [238]:
def predict_rating(userid, nn, simFunc):
    neighbor = nearest_neighbor_user2(userid, nn, simFunc)
    neighbor_id = [id for id, sim in neighbor]
    
    
    neighbor_movie = UM_matrix_ds.loc[neighbor_id].dropna(1, how='all',thresh= 1)
    neighbor_dict = (dict(neighbor))
    ret = []
    
    for movieId, row in neighbor_movie.iteritems():
        jsum, wsum = 0, 0
        for v in row.dropna().iteritems():
            sim = neighbor_dict.get(v[0], 0)
            jsum += sim
            wsum += (v[1]*sim)
        ret.append([movieId, wsum/jsum])
    return ret

In [241]:
def nearest_neighbor_user2(user, topN, simFunc):
    ul = UM_matrix_ds.loc[user].dropna()
    ratedIndex = ul.index
    nn = {}
    
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = []
        interSectionU2 = []
        if uid == user: continue
            
        for i in ratedIndex:
            if not math.isnan(row[i]):
                interSectionU1.append(ul[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
            
        sim = simFunc(interSectionU1, interSectionU2)
        if not math.isnan(sim): nn[uid] = sim

            
    return sorted(nn.items(), key= itemgetter(1))[:-(topN+1):-1]

## [과제 2-b] Cosine

In [250]:
result = []
for i in range(10):
    reviewNo = int(top10.iloc[i].reviewNo)
    movieId = int(top10.iloc[i].movieId)
    predict = predict_rating(reviewNo, 100, distance_cosine)
    
    for movie in predict:
        if movieId == movie[0]:
            result.append([int(reviewNo), int(movieId), movie[1]])

resultdf = pd.DataFrame(result, columns=['userId','movieId', 'rating'])
print('=====Cosine=====')
print(resultdf)

=====Cosine=====
     userId  movieId    rating
0  15771936   145162  5.796201
1  15771961   161967  8.595640
2  15771934   163788  9.150525
3  15771998   180399  7.000000
4  15772036    86507  8.561077
5  15771976   180399  7.000000
6  15771977   157297  5.800920


In [243]:
realdata_rating = []
for userid in resultdf['userId']:
    realdata_rating.append(float(top10[top10['reviewNo']==userid]['rating']))

resultdata_rating = resultdf.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating, resultdata_rating)
print("\nError Rate(Absolute) : ", error_rate_absol)
print("Error Rate(Squared) : ", error_rate_squared)


Error Rate(Absolute) :  1.798704122760153
Error Rate(Squared) :  5.6478653086387975


## [과제 2-b] Euclidean

In [248]:
result = []
for i in range(10):
    reviewNo = int(top10.iloc[i].reviewNo)
    movieId = int(top10.iloc[i].movieId)
    predict = predict_rating(reviewNo, 100, distance_euclidean)
    
    for movie in predict:
        if movieId == movie[0]:
            result.append([int(reviewNo), int(movieId), movie[1]])

resultdf = pd.DataFrame(result, columns=['userId','movieId', 'rating'])
print('=====Euclidean=====')
print(resultdf)

=====Euclidean=====
     userId  movieId    rating
0  15771936   145162  4.649008
1  15771961   161967  9.263031
2  15771934   163788  8.956179
3  15771998   180399  7.000000
4  15772036    86507  8.054289
5  15771976   180399  7.000000
6  15771977   157297  6.588583


In [245]:
realdata_rating = []
for userid in resultdf['userId']:
    realdata_rating.append(float(top10[top10['reviewNo']==userid]['rating']))

resultdata_rating = resultdf.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating, resultdata_rating)
print("\nError Rate(Absolute) : ", error_rate_absol)
print("Error Rate(Squared) : ", error_rate_squared)


Error Rate(Absolute) :  1.6966357708958737
Error Rate(Squared) :  6.0152917257030625


## [과제 2-b] Correlation

In [263]:
result = []
for i in range(10):
    reviewNo = int(top10.iloc[i].reviewNo)
    movieId = int(top10.iloc[i].movieId)
    predict = predict_rating(reviewNo, 100, distance_correlation)
    
    for movie in predict:
        if movieId == movie[0]:
            result.append([int(reviewNo), int(movieId), movie[1]])

resultdf = pd.DataFrame(result, columns=['userId','movieId', 'rating'])
print('=====Correlation=====')
print(resultdf)

/home/khz/anaconda3/envs/nack/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


=====Correlation=====
     userId  movieId     rating
0  15771936   145162   5.433217
1  15771961   161967  10.715807
2  15771934   163788  15.406481
3  15771998   180399   7.000000
4  15772036    86507  11.304849
5  15771976   180399   7.000000
6  15771977   157297   0.954683


In [264]:
realdata_rating = []
for userid in resultdf['userId']:
    realdata_rating.append(float(top10[top10['reviewNo']==userid]['rating']))

resultdata_rating = resultdf.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating, resultdata_rating)
print("\nError Rate(Absolute) : ", error_rate_absol)
print("Error Rate(Squared) : ", error_rate_squared)


Error Rate(Absolute) :  1.8436674517673026
Error Rate(Squared) :  11.258077133072339
